<a href="https://colab.research.google.com/github/leonas5555/colab/blob/main/BERT_based_synthesized_graph_for_Neural_Structured_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

create IMDB dataset text embeddings and build a graph based on these embeddings using the BERT model

In [1]:
!pip install transformers networkx torchtext portalocker

In [4]:
from torchtext.datasets import IMDB
from transformers import BertTokenizer, BertModel
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import portalocker
import torch

train_iter, test_iter = IMDB(root='.data')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')



Generate embeddings for the text data. Tokenize the text and pass it through the BERT model to get the embeddings. !BERT has input tokens limit of 512.

In [ ]:
def get_embedding(text, max_length=512):
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=max_length)).unsqueeze(0)  # Batch size 1
    with torch.no_grad(): outputs = model(input_ids)
    last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
    return last_hidden_states.mean(dim=1).numpy()  # Average the embeddings across the tokens

embeddings = [get_embedding(text) for _ , text in train_iter] # call get_embedding with the updated parameters

Build a graph based on the embeddings. Use a graph library like networkx to create a graph. Nodes can represent the text samples, and edges can be created based on the similarity between their embeddings using cosine similarity.

In [ ]:
G = nx.Graph()
similarity_matrix = cosine_similarity(embeddings)

for i in range(len(embeddings)):
  for j in range(i + 1, len(embeddings)):
    similarity = similarity_matrix[i][j]
    if similarity > threshold:  # Set a threshold for edge creation
      G.add_edge(i, j, weight=similarity)